# Titanic Dataset

One dataset we all data science / machine learning enthusiast come across is dealing with the Titanic dataset.A lot of learners have dealt with this dataset in different ways. After going through a few books and learning about different classification algorithms I decided why don't I myself try my hand on this. I also wanted to answer the oldest question about the whole Titanic disaster. Yes you got that right. Would Jack have survived irrespective of how big the plank was? Did James Cameron kill off Jack to make the movie a block buster?

# Lets find out

In [256]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [257]:
t_train = pd.read_csv("train.csv")
t_test = pd.read_csv("test.csv")
y_test = pd.read_csv("gender_submission.csv")
y_test

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


The dataset provides us with the following information
- **Survived** says whether the passenger survived or not i.e 1 or 0 respectively
- **pclass** is the Ticket Class
- **sex** indicates whether male or female
- **sibsp** indicates number of sibling or spouse aboard
- **parch** is the number of parents and children
- **ticket** is the ticket number
- **fare** is the passenger fare
- **cabin** is the cabin number
- **embarked** is from where they began the journey i.e Cherboug / Queenstown / Southampton

Time to analyse the data and find out all we can


In [258]:
t_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [259]:
l = t_train.columns
for x in l:
    nan_per = (t_train[x].isnull().sum()/len(t_train[x])*100).round(2)
    print("{}:\t{}%".format(x,nan_per))

PassengerId:	0.0%
Survived:	0.0%
Pclass:	0.0%
Name:	0.0%
Sex:	0.0%
Age:	19.87%
SibSp:	0.0%
Parch:	0.0%
Ticket:	0.0%
Fare:	0.0%
Cabin:	77.1%
Embarked:	0.22%


We can see that there is a lot of data missing in the Cabin column i.e about 77% of the data is missing. Due to this reason we shall not use the cabin column for our predictions. The age and Embarked column don't have much missing data and those missing places can be filled or deleted

In [260]:
median = t_train["Age"].median()
t_train["Age"].fillna(median,inplace = True)
t_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [261]:
t_train["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [262]:
t_train = t_train.drop("Cabin", axis = 1)
t_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


There is a lot of difference between the min and max value lets see more about the fare value

In [263]:
t_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


We can see that the Age has a minimum value of 0.42 Which I believe is put down incorrectly hence its better to correct it

From the data above I see that there are a lot of ages below 1 which I believe is a error. Since all these people survived I believe there supposed to be Senior citizens and were given the chance to move to safety first hence I would multiply these values by 100

In [264]:
def age_filter(t):
    filter_Age = t["Age"]<1
    age = t["Age"][filter_Age]
    for x in age.index:
        t.iloc[x,5] = t.iloc[x,5]*100
    t= t.drop(t_train.iloc[:,12:],axis = 1)
    t = t.drop(t_train.iloc[:,10:11],axis = 1)

In [265]:
age_filter(t_train)
age_filter(t_test)
t_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


Lets Bin the age group column into bin of 10's. Hopefully this helps

In [266]:
t_train["AgeBin"] = t_train["Age"] // 10

t_test["AgeBin"] = t_test["Age"] // 10
t_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,AgeBin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,2.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,3.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,2.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,3.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,2.0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,1.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S,2.0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,2.0


In [267]:
X =t_train[["Pclass","Sex","Fare","AgeBin","Embarked"]]
y = t_train["Survived"]


In [268]:
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values


In [269]:
from sklearn.impute import SimpleImputer

In [276]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
num =["Fare","AgeBin"]
cat = ["Pclass","Sex","Embarked"]
cat_pipeline = Pipeline([
    ("Selector", DataFrameSelector(cat)),
    ("Imputer",SimpleImputer(strategy = "most_frequent")),
    ("Encoder",OneHotEncoder(sparse=False))
])
num_pipeline = Pipeline([
    ("Selector",DataFrameSelector(num)),
    ("Imputer",SimpleImputer(strategy = "median")),
    ("std_scalar", StandardScaler())
])

In [277]:
from sklearn.pipeline import FeatureUnion
full_pipeline = FeatureUnion(transformer_list = [
    ("Numerical", num_pipeline),
    ("Categories", cat_pipeline)
])

In [278]:
titanic_prepared = full_pipeline.fit_transform(X)
titanic_prepared.shape

(891, 10)

In [279]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(titanic_prepared, y)
titanic_test = full_pipeline.fit_transform(t_test)
y_test_hat = model.predict(titanic_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test["Survived"],y_test_hat, normalize = True) * 100 

96.41148325358851

In [280]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
forest_clf = RandomForestClassifier( random_state=42)
forest_scores = cross_val_score(forest_clf,titanic_prepared, y, cv=10)
(forest_scores.mean())*100

80.70287141073658